In [33]:
import funcy
import difflib
import Levenshtein as lev
import pandas as pd
import numpy as np
import re
import string
from fuzzywuzzy import fuzz
from os.path import join


RAW_DATA = '/Users/lrraymond13/MIT/Furman_RA_2016/journal_analysis/Data/RawData'
INCITES = 'InCitesJournalGrid.csv'
JOURNAL_KEY = 'Furman_Journal_Key2016.csv'

In [ ]:
# 1. Check that the spellings in the JOURNAL KEY uniquely identify a journal. 
# If there are multiple spellings for the same journal, adjust the Journal ID. 
# Specifically, in column AdjustedJournalID enter the same number from column JournalID corresponding
# to one of the spellings for all alternative spellings. When in doubt, leave as separate journals 
# i.e. with separate JournalID numbers.

# 3. Pair the spelling from InCites with the one in the attached file, for each identified unique AdjustedJournalID. 

# 4. Add in the Excel file the InCites spelling, along with Total Cites, Journal Impact Factor and Eigenfactor 
# from InCites.

In [ ]:
# import journal key file, check each spelling is unique
jk_df = pd.read_csv(join(RAW_DATA, JOURNAL_KEY))
jk_df.columns = map(lambda x: '_'.join(x.split(' ')).upper(), jk_df.columns)
print jk_df.columns

In [4]:
# Check the length of unique pub titles is same as length of series
def check_unique(uniq_ser):
    # check if dup values in unique series
    unique_titles = uniq_ser.unique()
    return len(unique_titles) == uniq_ser.shape[0]

In [5]:
check_unique(jk_df.loc[:, 'PUBLICATION_TITLE'])

True

In [6]:
# Check if there are any modifications caused by changing journal to uppercase and stripping extraneous chars

In [7]:
# string cleaning functions
def trans_remov_punc(to_change, change_to):
    # removes specified punctuation using string maketrans (very fast, C lookups)
    #returns partially evaluated fnc
    trantab = string.maketrans(to_change, change_to)
    return funcy.func_partial(lambda x: x.translate(trantab))


def standardize_whitespace(pub_str):
    return ' '.join(filter(None, pub_str.split(' ')))


def remove_punc(pub_str):
    # function to remove punctuation
    nonelst = ' '*len(string.punctuation)
    fn = trans_remov_punc(string.punctuation, nonelst)
    new_str = fn(pub_str)
    # standardize spaces
    return standardize_whitespace(new_str)


def has_year(pub_str):
    # checks if string contains a year
    # each string should always have punctuation removed
    if isinstance(pub_str, str):
        pub_str = pub_str.split(' ')
    nums = [int(s) for s in pub_str if s.isdigit()]
    return any(map(lambda x: 1900 < x < 2018, nums))


def seperate_IEEE_IET(fnc_str):
# many strings are title, IEEE while in incites matching file formar is IEEE title. 
    if not isinstance(fnc_str, str):
        strs = ' '.join(fnc_str)
    strs = fnc_str.rsplit(', ', 1)
    return ' '.join(strs[::-1])


def remove_acronym(pub_str):
    # removes acronyms enclosed in parens ex: (SAS)
    re_match = " \(\S+\)"
    if not re.search(re_match, pub_str, re.IGNORECASE):
        return pub_str
    r = re.compile(r" \(\S+\)", re.IGNORECASE)
    new_str = r.sub(r'', pub_str)
    return new_str


def remove_year(pub_str):
    re_match = "\d{4}"
    if not re.search(re_match, pub_str, re.IGNORECASE):
        return pub_str
    r = re.compile(r"\d{4}", re.IGNORECASE)
    new_str = r.sub(r'', pub_str)
    return new_str


def clean_pubname(pub_str):
    if ', ' in pub_str:
        pub_str = seperate_IEEE_IET(pub_str)
    no_ac = remove_acronym(pub_str)
    # now remove punctuation
    no_pnc = remove_punc(no_ac)
    is_year = has_year(no_pnc)
    if is_year:
        # remove year
        no_pnc = remove_year(no_pnc)
    return standardize_whitespace(no_pnc)
    

def is_conf(pub_str, conf_strings=None):
    # checks if string appears ot be a conference
    if conf_strings is None:
        conf_strings = ['SYMPOSIUM', 'CONFERENCE',
                        'COLLOQUIUM', 'CONGRESS', 'ANNUAL', 'FORUM', 'WORKSHOP', 'SEMINAR']
    is_conf = any(map(lambda x: x in pub_str, conf_strings))
    # check if date exists in any of the string
    is_year = has_year(pub_str)
    if is_conf or is_year:
        return 1
    return 0  
    

In [8]:
clean_pubname('Service-Oriented Computing and Applications (SOCA), 2009 IEEE International Conference on')

'IEEE International Conference on Service Oriented Computing and Applications'

In [9]:
jk_df['CLEAN_PUB'] = jk_df['PUBLICATION_TITLE'].apply(clean_pubname)
check_unique(jk_df.loc[:, 'CLEAN_PUB'])

False

In [10]:
# check which are not the same
dups = jk_df[jk_df.duplicated(subset=['CLEAN_PUB'], keep=False)]

In [11]:
dups

,PUBLICATION_TITLE,JOURNALID,ADJUSTEDJOURNALID,INCITES_SPELLING,CLEAN_PUB
16,"3D Imaging (IC3D), 2012 International Conferen...",17,NaN,NaN,International Conference on 3D Imaging
17,"3D Imaging (IC3D), 2013 International Conferen...",18,NaN,NaN,International Conference on 3D Imaging
22,"3D Systems Integration Conference (3DIC), 2010...",23,NaN,NaN,IEEE International 3D Systems Integration Conf...
23,"3D Systems Integration Conference (3DIC), 2011...",24,NaN,NaN,IEEE International 3D Systems Integration Conf...
24,"3D Systems Integration Conference (3DIC), 2013...",25,NaN,NaN,IEEE International 3D Systems Integration Conf...
25,"3D User Interfaces (3DUI), 2010 IEEE Symposium on",26,NaN,NaN,IEEE Symposium on 3D User Interfaces
26,"3D User Interfaces (3DUI), 2011 IEEE Symposium on",27,NaN,NaN,IEEE Symposium on 3D User Interfaces
27,"3D User Interfaces (3DUI), 2012 IEEE Symposium on",28,NaN,NaN,IEEE Symposium on 3D User Interfaces
28,"3D User Interfaces (3DUI), 2013 IEEE Symposium on",29,NaN,NaN,IEEE Symposium on 3D User Interfaces
29,"3D User Interfaces (3DUI), 2014 IEEE Symposium on",30,NaN,NaN,IEEE Symposium on 3D User Interfaces


In [19]:
# for each of the duplicates, adjust any subsequent journal ids
standard_dups = jk_df.loc[jk_df.duplicated(subset=['CLEAN_PUB'], keep='last'), ['CLEAN_PUB', 'JOURNALID']]
standard_dups.rename(columns={'JOURNALID': 'DEDUP_JOURNALID'}, inplace=True)
jk_df2 = pd.merge(
    left=jk_df, right=standard_dups.loc[:, ['CLEAN_PUB','DEDUP_JOURNALID']],
                  left_on='CLEAN_PUB', right_on='CLEAN_PUB', how='left')

In [21]:
jk_df2['CONF_FLAG'] = jk_df2['CLEAN_PUB'].apply(is_conf)

In [24]:
incites_df = pd.read_csv(join(RAW_DATA, INCITES), skiprows=1)
incites_df.columns = map(lambda x: '_'.join(x.split(' ')).upper(), incites_df.columns)

In [25]:
# standardize journal title to uppercase and remove punctuation
incites_df['CLEAN_PUB'] = incites_df['FULL_JOURNAL_TITLE'].apply(clean_pubname)

In [26]:
# merge the jk_df2 clean pub name with the incites df
exact_matches = pd.merge(
    left=incites_df, right=jk_df2, left_on='CLEAN_PUB', right_on='CLEAN_PUB', how='inner')

In [27]:
exact_matches

,RANK,FULL_JOURNAL_TITLE,TOTAL_CITES,JOURNAL_IMPACT_FACTOR,EIGENFACTOR_SCORE,CLEAN_PUB,PUBLICATION_TITLE,JOURNALID,ADJUSTEDJOURNALID,INCITES_SPELLING,DEDUP_JOURNALID,CONF_FLAG
0,602,IEEE Industrial Electronics Magazine,518,5.303,0.00265,IEEE Industrial Electronics Magazine,"Industrial Electronics Magazine, IEEE",8915,NaN,NaN,NaN,0
1,702,IEEE Transactions on Cybernetics,"2,246",4.943,0.01017,IEEE Transactions on Cybernetics,"Cybernetics, IEEE Transactions on",3358,NaN,NaN,NaN,0
2,728,IEEE Transactions on Neural Networks and Learn...,"12,919",4.854,0.02248,IEEE Transactions on Neural Networks and Learn...,"Neural Networks and Learning Systems, IEEE Tra...",11668,NaN,NaN,NaN,0
3,782,IEEE Transactions on Industrial Informatics,"3,779",4.708,0.01639,IEEE Transactions on Industrial Informatics,"Industrial Informatics, IEEE Transactions on",8970,NaN,NaN,NaN,0
4,1266,IEEE Journal of Photovoltaics,"2,469",3.736,0.00960,IEEE Journal of Photovoltaics,"Photovoltaics, IEEE Journal of",12495,NaN,NaN,NaN,0
5,1271,IEEE Transactions on Sustainable Energy,"2,149",3.727,0.01171,IEEE Transactions on Sustainable Energy,"Sustainable Energy, IEEE Transactions on",15045,NaN,NaN,NaN,0
6,1330,IEEE Computational Intelligence Magazine,634,3.647,0.00165,IEEE Computational Intelligence Magazine,"Computational Intelligence Magazine, IEEE",2384,NaN,NaN,NaN,0
7,1738,IEEE Transactions on Smart Grid,"4,753",3.19,0.02922,IEEE Transactions on Smart Grid,"Smart Grid, IEEE Transactions on",14442,NaN,NaN,NaN,0
8,1796,IEEE Journal of Emerging and Selected Topics i...,473,3.129,0.00305,IEEE Journal of Emerging and Selected Topics i...,Emerging and Selected Topics in Power Electron...,4809,NaN,NaN,NaN,0
9,1950,IEEE Circuits and Systems Magazine,550,3,0.00142,IEEE Circuits and Systems Magazine,"Circuits and Systems Magazine, IEEE",1640,NaN,NaN,NaN,0


In [100]:
not_match = jk_df2.loc[~jk_df2['CLEAN_PUB'].isin(exact_matches['CLEAN_PUB']), :]
not_match.set_index(keys='CLEAN_PUB', drop=True, inplace=True)

not_match_incites = incites_df.loc[~incites_df['CLEAN_PUB'].isin(exact_matches.index), :]
not_match_incites.set_index(keys='CLEAN_PUB', drop=True, inplace=True)

In [101]:
def difflib_get_close_matches(key_str, match_index):
    matches = difflib.get_close_matches(key_str, match_index, n=2, cutoff=0.8)
    if len(matches) == 0:
        return np.nan 
    print key_str, matches[0]
    return matches[0]

In [102]:
def get_close_matches(key_str, match_index):
    print key_str
    string_sim = map(lambda (num, x): (num, lev.ratio(x, key_str)), enumerate(match_index))
    max_str = sorted(string_sim, key=funcy.second)[0]
    if max_str[1] < .8:
        return np.nan 
    print key_str, match_index[max_str[0]]
    return match_index[max_str[0]]


In [99]:
not_match_incites

,CLEAN_PUB,RANK,FULL_JOURNAL_TITLE,TOTAL_CITES,JOURNAL_IMPACT_FACTOR,EIGENFACTOR_SCORE
0,CA A CANCER JOURNAL FOR CLINICIANS,1,CA-A CANCER JOURNAL FOR CLINICIANS,"20,488",131.723,0.06261
1,NEW ENGLAND JOURNAL OF MEDICINE,2,NEW ENGLAND JOURNAL OF MEDICINE,"283,525",59.558,0.68563
2,NATURE REVIEWS DRUG DISCOVERY,3,NATURE REVIEWS DRUG DISCOVERY,"25,460",47.12,0.06294
3,LANCET,4,LANCET,"195,553",44.002,0.40817
4,NATURE BIOTECHNOLOGY,5,NATURE BIOTECHNOLOGY,"48,650",43.113,0.15771
5,NATURE REVIEWS IMMUNOLOGY,6,NATURE REVIEWS IMMUNOLOGY,"31,545",39.416,0.08760
6,NATURE MATERIALS,7,NATURE MATERIALS,"72,306",38.891,0.20799
7,NATURE REVIEWS MOLECULAR CELL BIOLOGY,8,NATURE REVIEWS MOLECULAR CELL BIOLOGY,"36,784",38.602,0.09969
8,NATURE,9,NATURE,"627,846",38.138,1.44762
9,Annual Review of Astronomy and Astrophysics,10,Annual Review of Astronomy and Astrophysics,"9,000",37.846,0.02021


In [103]:
not_match['DIFF_FUZZ_MATCH'] = not_match.index.map(lambda x: difflib_get_close_matches(x, not_match_incites.index))

IEEE Transactions on Acoustics Speech and Signal Processing IEEE Transactions on Audio Speech and Language Processing
ISAUHC 06 International Symposium on Ad Hoc and Ubiquitous Computing International Journal of Ad Hoc and Ubiquitous Computing
IEEE International Symposium on Antennas and Propagation International Journal of Antennas and Propagation
International Symposium on Antennas and Propagation International Journal of Antennas and Propagation
International Symposium on Antennas and Propagation International Journal of Antennas and Propagation
Applied Physics Letters Applied Physics Reviews
IEEE Conference on Automation Science and Engineering IEEE Transactions on Automation Science and Engineering
IEEE Conference on Automation Science and Engineering IEEE Transactions on Automation Science and Engineering
IEEE International Conference on Automation Science and Engineering IEEE Transactions on Automation Science and Engineering
IEEE International Conference on Automation Science a

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [104]:
# test = not_match.iloc[:300, :].index.map(lambda x: difflib_get_close_matches(x, not_match_incites.index))
# not_match['FUZZ_MATCH'] = not_match.index.map(lambda x: get_close_matches(x, not_match_incites.index))

In [106]:
not_match.dropna(subset=['DIFF_FUZZ_MATCH'])

,PUBLICATION_TITLE,JOURNALID,ADJUSTEDJOURNALID,INCITES_SPELLING,DEDUP_JOURNALID,CONF_FLAG,DIFF_FUZZ_MATCH
CLEAN_PUB,,,,,,,
IEEE Transactions on Acoustics Speech and Signal Processing,"Acoustics, Speech and Signal Processing, IEEE ...",78,NaN,NaN,NaN,0,IEEE Transactions on Audio Speech and Language...
ISAUHC 06 International Symposium on Ad Hoc and Ubiquitous Computing,"Ad Hoc and Ubiquitous Computing, 2006. ISAUHC ...",112,NaN,NaN,NaN,0,International Journal of Ad Hoc and Ubiquitous...
IEEE International Symposium on Antennas and Propagation,"Antennas and Propagation (APSURSI), 2011 IEEE ...",668,NaN,NaN,NaN,0,International Journal of Antennas and Propagation
International Symposium on Antennas and Propagation,"Antennas and Propagation (ISAP), 2012 Internat...",674,NaN,NaN,674.0,0,International Journal of Antennas and Propagation
International Symposium on Antennas and Propagation,"Antennas and Propagation (ISAP), 2014 Internat...",675,NaN,NaN,674.0,0,International Journal of Antennas and Propagation
Applied Physics Letters,Applied Physics Letters,893,NaN,NaN,NaN,0,Applied Physics Reviews
IEEE Conference on Automation Science and Engineering,"Automation Science and Engineering (CASE), 201...",1123,NaN,NaN,1123.0,0,IEEE Transactions on Automation Science and En...
IEEE Conference on Automation Science and Engineering,"Automation Science and Engineering (CASE), 201...",1124,NaN,NaN,1123.0,0,IEEE Transactions on Automation Science and En...
IEEE International Conference on Automation Science and Engineering,"Automation Science and Engineering (CASE), 201...",1125,NaN,NaN,1125.0,0,IEEE Transactions on Automation Science and En...


In [107]:
# for those with a first column that is not none, merge with incites
not_match_incites.reset_index(drop=False, inplace=True)
fuzz_match = not_match.dropna(subset=['DIFF_FUZZ_MATCH'])
fuzz_match['FUZZY_MATCH'] = 1
fuzz_match_key = pd.merge(
    left=fuzz_match, right=not_match_incites, left_on='DIFF_FUZZ_MATCH', 
    right_on='CLEAN_PUB', how='inner')


/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [112]:
# combine fuzz_matches and exact matches
full_matches = pd.concat([exact_matches, fuzz_match_key], axis=0)

In [115]:
full_matches['INCITES_SPELLING'] = full_matches['FULL_JOURNAL_TITLE']
full_matches['ADJUSTEDJOURNALID'] = full_matches['DEDUP_JOURNALID']
del full_matches['DEDUP_JOURNALID'], full_matches['FULL_JOURNAL_TITLE']

In [126]:
full_matches.to_csv(join(RAW_DATA, 'journal_matches.csv'))

In [119]:
not_match

,PUBLICATION_TITLE,JOURNALID,ADJUSTEDJOURNALID,INCITES_SPELLING,DEDUP_JOURNALID,CONF_FLAG,DIFF_FUZZ_MATCH
CLEAN_PUB,,,,,,,
Proceedings of the First Joint Engineering in Medicine and Biology 21st Annual Conf and the Annual Fall Meeting of the Biomedical Engineering Soc BMES EMBS Conference,"[Engineering in Medicine and Biology, 1999. 21...",1,NaN,NaN,NaN,0,NaN
February D9,"1703/D9, February 2012",2,NaN,NaN,NaN,0,NaN
Ultrasonics Symposium,1980 Ultrasonics Symposium,3,NaN,NaN,NaN,0,NaN
EOS ESD 02 Electrical Overstress Electrostatic Discharge Symposium,2002 Electrical Overstress/Electrostatic Disch...,4,NaN,NaN,NaN,0,NaN
Mobile Networking for Vehicular Environments,2007 Mobile Networking for Vehicular Environments,5,NaN,NaN,NaN,0,NaN
International Conference on Signal Processing Systems,2009 International Conference on Signal Proces...,6,NaN,NaN,NaN,0,NaN
IEEE AIAA 25th Digital Avionics Systems Conference,"25th Digital Avionics Systems Conference, 2006...",7,NaN,NaN,NaN,0,NaN
EOS ESD 29th Electrical Overstress Electrostatic Discharge Symposium,29th Electrical Overstress/Electrostatic Disch...,8,NaN,NaN,NaN,0,NaN
Proceedings First International Symposium on 3D Data Processing Visualization and Transmission,3D Data Processing Visualization and Transmiss...,9,NaN,NaN,NaN,0,NaN


In [120]:
# get not matched from csv
not_matched2 = not_match[~not_match.index.isin(full_matches['CLEAN_PUB'])]
not_matched_incites2 = not_match_incites[~not_match_incites['FULL_JOURNAL_TITLE'].isin(full_matches['INCITES_SPELLING'])]

In [123]:
not_matched2.reset_index(drop=False, inplace=True)
not_matched2.to_csv(join(RAW_DATA, 'journals_not_matched.csv'))

In [125]:
not_matched_incites2.to_csv(join(RAW_DATA, 'incites_not_matched.csv'))

In [66]:
jk_df2[jk_df2['CONF_FLAG']==1].to_csv(join(RAW_DATA, 'conferences.csv'))
jk_df2[jk_df2['CONF_FLAG']==0].to_csv(join(RAW_DATA, 'journals_raw.csv'))